In [66]:
class BluePrint:
    def __init__(self, oreRobotOreCost, clayRobotOreCost, obsidianRobotOreCost, obsidianRobotClayCost, geodeRobotObsidianCost, geodeRobotOreCost):
        self.oreRobotOreCost = oreRobotOreCost
        self.clayRobotOreCost = clayRobotOreCost
        self.obsidianRobotOreCost = obsidianRobotOreCost
        self.obsidianRobotClayCost = obsidianRobotClayCost
        self.geodeRobotObsidianCost = geodeRobotObsidianCost
        self.geodeRobotOreCost = geodeRobotOreCost
    
    def __str__(self):
        return f'{self.oreRobotOreCost}, {self.clayRobotOreCost}, {self.obsidianRobotOreCost}, {self.obsidianRobotClayCost}, {self.geodeRobotObsidianCost}, {self.geodeRobotOreCost}, '

In [67]:
with open("test-input.txt") as f:
    lines = list(map(lambda x: x.strip(), f.readlines()))

blueprints = []

for line in lines:
    sentences = line.split(".")
    oreRobotOreCost  = int(sentences[0].split(" ")[-2]) 
    clayRobotOreCost  = int(sentences[1].split(" ")[-2])
    obsidianRobotOreCost  = int(sentences[2].split(" ")[-5])
    obsidianRobotClayCost  = int(sentences[2].split(" ")[-2])
    geodeRobotOreCost  = int(sentences[3].split(" ")[-5])
    geodeRobotObsidianCost  = int(sentences[3].split(" ")[-2])

    blueprints+= [BluePrint(oreRobotOreCost,
    clayRobotOreCost,
    obsidianRobotOreCost,
    obsidianRobotClayCost,
    geodeRobotObsidianCost,
    geodeRobotOreCost)]

In [68]:
class Scenario:
    def __init__(self, blueprint):
        self.time = 24
        self.ore = 0
        self.clay = 0
        self.obsidian = 0
        self.geode = 0

        self.oreRobot = 1
        self.clayRobot = 0
        self.obsidianRobot = 0
        self.geodeRobot = 0

        self.blueprint = blueprint

    def __farmResourcesRound__(self):
        self.ore += self.oreRobot
        self.clay += self.clayRobot
        self.obsidian += self.obsidianRobot
        self.geode += self.geodeRobot
        self.time -= 1

    def __addOreRobotRound__(self) -> bool:
        if self.ore >= self.blueprint.oreRobotOreCost:
            self.ore -= self.blueprint.oreRobotOreCost
            self.__farmResourcesRound__()
            self.oreRobot += 1
            return True
        self.__farmResourcesRound__()
        return False

    def __addClayRobotRound__(self) -> bool:
        if self.ore >= self.blueprint.clayRobotOreCost:
            self.ore -= self.blueprint.clayRobotOreCost
            self.__farmResourcesRound__()
            self.clayRobot += 1
            return True
        self.__farmResourcesRound__()
        return False

    def __addObsidianRobotRound__(self) -> bool:
        if self.ore >= self.blueprint.obsidianRobotOreCost and self.clay >= self.blueprint.obsidianRobotClayCost:
            self.ore -= self.blueprint.obsidianRobotOreCost
            self.clay -= self.blueprint.obsidianRobotClayCost
            self.__farmResourcesRound__()
            self.obsidianRobot += 1
            return True
        self.__farmResourcesRound__()
        return False
    
    def __addGeodeRobotRound__(self) -> bool:
        if self.ore >= self.blueprint.geodeRobotOreCost and self.obsidian >= self.blueprint.geodeRobotObsidianCost:
            self.ore -= self.blueprint.geodeRobotOreCost
            self.obsidian -= self.blueprint.geodeRobotObsidianCost
            self.__farmResourcesRound__()
            self.geodeRobot += 1
            return True
        self.__farmResourcesRound__()
        return False

In [69]:
def generateRoutes():
    routes=[""]

    for i in range(0,13):
        newRoutes = []
        for route in routes:
            newRoutes.append(route+"I")
            newRoutes.append(route+"C")
            if "C" in route:
                newRoutes.append(route+"O")
            if "O" in route:
                newRoutes.append(route+"G")
        routes = newRoutes
    return list(filter(lambda route: "G" in route and route[-1] == "G", routes))

def simulateRoute(route, blueprint):
    scenario = Scenario(blueprint=blueprint)
    routeSteps = [*route]

    nextRobot = routeSteps.pop(0)
    while len(routeSteps) or scenario.time > 0:
        robotCreated = False
        if(scenario.time > 0):
            if nextRobot == "I": robotCreated = scenario.__addOreRobotRound__()
            elif nextRobot == "C": robotCreated = scenario.__addClayRobotRound__()
            elif nextRobot == "O": robotCreated = scenario.__addObsidianRobotRound__()
            elif nextRobot == "G": robotCreated = scenario.__addGeodeRobotRound__()
            if robotCreated == True and len(routeSteps) > 0: nextRobot = routeSteps.pop(0)
            #if len(routeSteps)==0: print("ROUTE TOO SHORT!")
        else:
            break
    return scenario.geode

routes = generateRoutes()
len(routes)

2794155

In [70]:
geodeCounts = []
for blueprint in blueprints:
    print(blueprint)
    blueprintGeodeCounts = list(map(lambda route: simulateRoute(route,blueprint), routes))
    print(max(blueprintGeodeCounts))
    geodeCounts.append(max(blueprintGeodeCounts))
geodeCounts

4, 2, 3, 14, 7, 2, 


KeyboardInterrupt: 